In [105]:
import pandas as pd
import numpy as np

## `map`

## ``transform``

In [2]:
import pandas as pd
import numpy as np

# Crear un DataFrame de ejemplo
df = pd.DataFrame({
    'A': ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],
    'B': ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
    'C': [1, 2, 3, 4, 5, 6, 7, 8],
    'D': [9,10,11,12,13,14,15,16]
})

df

,A,B,C,D
0,foo,one,1,9
1,bar,one,2,10
2,foo,two,3,11
3,bar,three,4,12
4,foo,two,5,13
5,bar,two,6,14
6,foo,one,7,15
7,foo,three,8,16


In [107]:
# grouby a dos columnas
a = df.groupby(['A', 'B']).sum()
a

C   D
A   B           
bar one    2  10
    three  4  12
    two    6  14
foo one    8  24
    three  8  16
    two    8  24

### Ejemplo incorrecto

In [108]:
# Definir una función personalizada que toma un DataFrame (o Serie) y dos parámetros
def multiply_and_add(x, multiplier, adder):
    return (x * multiplier) + adder

**No tine sentido**, es indeferente que .groupby('A') este. Seria igual selecionar las columnas especificas y aplicar el transform directamente.

Se selcionan solo las columnas "C" y "D" dado que las operaciones de add (suma) no estan permitidad con cadenas. Si se cambia la suma por mutiplicacion se puede agregar la columna "B"

In [109]:
result = df.groupby('A')[["C", "D"]].transform(multiply_and_add, multiplier=2, adder=3)



In [110]:
result = df[["C", "D"]].transform(multiply_and_add, multiplier=2, adder=3)
print(result)

    C   D
0   5  21
1   7  23
2   9  25
3  11  27
4  13  29
5  15  31
6  17  33
7  19  35


### Ejemplo correcto

hace una especie de concat

In [111]:
result = df.groupby('A').transform('sum')
result

C:\Users\lufer\AppData\Local\Temp\ipykernel_25184\2451963318.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  result = df.groupby('A').transform('sum')


,C,D
0,24,64
1,12,36
2,24,64
3,12,36
4,24,64
5,12,36
6,24,64
7,24,64


# Combining / comparing / joining / merging

## Join

[link](https://www.w3resource.com/pandas/dataframe/dataframe-join.php)

In [112]:
df = pd.DataFrame({'key': ['K0', 'K2', 'K3', 'K4', 'K5', 'K6'],
                   'X': ['X0', 'X2', 'X3', 'X4', 'X5', 'X6']})
df

,key,X
0,K0,X0
1,K2,X2
2,K3,X3
3,K4,X4
4,K5,X5
5,K6,X6


In [113]:
other = pd.DataFrame({'key': ['K0', 'K2', 'K3'],
                      'Y': ['Y0', 'Y2', 'Y3']})
other

,key,Y
0,K0,Y0
1,K2,Y2
2,K3,Y3


Se combina a travez de indices y apra columnas repetidas se usa sufijos

- lsuffix: "sufijo" a agregar al nombre de la columna izquierda
- rsuffix: "sufijo" a agregar al nombre de la columna derecha
- La union se realiza por defecto (how='left') por la interseccion de los **indices**

In [114]:
df.join(other, lsuffix='_caller', rsuffix='_other')

,key_caller,X,key_other,Y
0,K0,X0,K0,Y0
1,K2,X2,K2,Y2
2,K3,X3,K3,Y3
3,K4,X4,NaN,NaN
4,K5,X5,NaN,NaN
5,K6,X6,NaN,NaN


Tambien opermite la combinacion de otras colunnas ademas por indices [ver](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html#pandas.DataFrame.join)

## Merge

- Si una columna tinen multiples valores repetidos se conservan todos los valores.

In [115]:
data1 = pd.DataFrame({
    'ID': ['101', '102','103','104', '105', '104', '105' ],
    'x1': ['1', '2', '3', '4', '5', '8', '9'],
    'x2': ['a', 'b', 'c', 'd', 'e', 'f', 'g'],
    'x3': ['16', '15', '14', '13', '12', '11', '10']}
)

data1

,ID,x1,x2,x3
0,101,1,a,16
1,102,2,b,15
2,103,3,c,14
3,104,4,d,13
4,105,5,e,12
5,104,8,f,11
6,105,9,g,10


In [116]:
data2 = pd.DataFrame({
    'ID': ['104', '105','106','107', '104'],
    'y1': ['x', 'y', 'x', 'y', 'x'],
    'y2': ['8', '6', '4', '2', '9'],}
)
data2

,ID,y1,y2
0,104,x,8
1,105,y,6
2,106,x,4
3,107,y,2
4,104,x,9


Cuando se utiliza la función `merge()` en pandas, se combinan dos DataFrames basándose en una o varias columnas comunes (llamadas claves). El procedimiento básico es el siguiente:

1. **Identificar las claves**: Las claves son las columnas que se utilizarán para combinar los DataFrames. Si no se especifica ninguna clave, pandas intentará usar las columnas con nombres comunes en ambos DataFrames.

2. **Combinar los DataFrames**: pandas buscará filas en ambos DataFrames que tengan el mismo valor en las columnas clave y las combinará en una sola fila en el nuevo DataFrame.

3. **Ordenar las columnas**: Por defecto, pandas ordenará las columnas en el DataFrame resultante en el siguiente orden:
    - Primero, todas las columnas clave que se utilizaron para el merge.
    - Luego, las columnas restantes de `data1` (el DataFrame en el que se llamó a la función `merge()`).
        - se identifica el primer identificador de la columna clave deacuerdo a **how** y luego se recorre el otro data2 y se bueca coincidencias por identificador una vez termiando se busca si existe en data1 el mismo identificador y se hace lo mismo.
    - Finalmente, las columnas restantes de `data2` (el DataFrame que se pasó como argumento a la función `merge()`).

In [117]:
data1.merge(data2) # al no colocar el parametro on, se hace un merge por todas las columnas que tienen el mismo nombre, en este caso ID y se hace un merge de tipo inner

,ID,x1,x2,x3,y1,y2
0,104,4,d,13,x,8
1,104,4,d,13,x,9
2,104,8,f,11,x,8
3,104,8,f,11,x,9
4,105,5,e,12,y,6
5,105,9,g,10,y,6


In [118]:
data2.merge(data1) 

,ID,y1,y2,x1,x2,x3
0,104,x,8,4,d,13
1,104,x,8,8,f,11
2,104,x,9,4,d,13
3,104,x,9,8,f,11
4,105,y,6,5,e,12
5,105,y,6,9,g,10


In [119]:
data_inner = pd.merge(data1, data2, on = "ID", how = "inner") # igual a data1.merge(data2)
data_inner

,ID,x1,x2,x3,y1,y2
0,104,4,d,13,x,8
1,104,4,d,13,x,9
2,104,8,f,11,x,8
3,104,8,f,11,x,9
4,105,5,e,12,y,6
5,105,9,g,10,y,6


In [120]:
data_outer = pd.merge(data1, data2, on = "ID", how = "outer")
data_outer

,ID,x1,x2,x3,y1,y2
0,101,1,a,16,NaN,NaN
1,102,2,b,15,NaN,NaN
2,103,3,c,14,NaN,NaN
3,104,4,d,13,x,8
4,104,4,d,13,x,9
5,104,8,f,11,x,8
6,104,8,f,11,x,9
7,105,5,e,12,y,6
8,105,9,g,10,y,6
9,106,NaN,NaN,NaN,x,4


In [121]:
data_left = pd.merge(data1, data2, on = "ID", how = "left")
data_left

,ID,x1,x2,x3,y1,y2
0,101,1,a,16,NaN,NaN
1,102,2,b,15,NaN,NaN
2,103,3,c,14,NaN,NaN
3,104,4,d,13,x,8
4,104,4,d,13,x,9
5,105,5,e,12,y,6
6,104,8,f,11,x,8
7,104,8,f,11,x,9
8,105,9,g,10,y,6


In [122]:
data_right = pd.merge(data1, data2, on = "ID", how = "right")
data_right

,ID,x1,x2,x3,y1,y2
0,104,4,d,13,x,8
1,104,8,f,11,x,8
2,105,5,e,12,y,6
3,105,9,g,10,y,6
4,106,NaN,NaN,NaN,x,4
5,107,NaN,NaN,NaN,y,2
6,104,4,d,13,x,9
7,104,8,f,11,x,9


``left_on`` y ``right_on`` son las columnas que se van a comparar para hacer el merge, es decir, si se quiere hacer un merge por la columna "A" de la tabla "A" y la columna "B" de la tabla "B" se debe hacer ``left_on="A"`` y ``right_on="B"``, estas columnas no necesariamente deben tener el mismo nombre, pero si deben tener el mismo tipo de dato y al menos un valor en común. Va retornar un dataframe con las columnas de ambas tablas.

In [123]:
# ejemplo left_on y right_on
pd.merge(data1, data2, left_on='ID', right_on='ID') 

,ID,x1,x2,x3,y1,y2
0,104,4,d,13,x,8
1,104,4,d,13,x,9
2,104,8,f,11,x,8
3,104,8,f,11,x,9
4,105,5,e,12,y,6
5,105,9,g,10,y,6


In [124]:
data_inner = pd.merge(data1, data2, on = "ID", how = "inner")
data_inner

,ID,x1,x2,x3,y1,y2
0,104,4,d,13,x,8
1,104,4,d,13,x,9
2,104,8,f,11,x,8
3,104,8,f,11,x,9
4,105,5,e,12,y,6
5,105,9,g,10,y,6


In [125]:
# ejemplo left_on y right_on
pd.merge(data2, data1, left_on='ID', right_on='ID') 

,ID,y1,y2,x1,x2,x3
0,104,x,8,4,d,13
1,104,x,8,8,f,11
2,104,x,9,4,d,13
3,104,x,9,8,f,11
4,105,y,6,5,e,12
5,105,y,6,9,g,10


In [126]:
# DataFrame de productos
df_productos = pd.DataFrame({
    'ID_Producto': ['P1', 'P2', 'P3', 'P4'],
    'Nombre': ['Manzana', 'Banana', 'Cereza', 'Durazno'],
    'Precio': [1.00, 0.50, 0.75, 1.25]
})

# DataFrame de ventas
df_ventas = pd.DataFrame({
    'ID_Producto': ['P1', 'P2', 'P2', 'P3', 'P4', 'P4', 'P4'],
    'Cantidad': [10, 5, 7, 15, 10, 10, 5]
})


## Concat

In [3]:
df_productos = pd.DataFrame({
    'ID_Producto': ['P1', 'P2', 'P3', 'P4'],
    'Nombre': ['Manzana', 'Banana', 'Cereza', 'Durazno'],
    'Precio': [1.00, 0.50, 0.75, 1.25]
})

df_ventas = pd.DataFrame({
    'ID_Producto': ['P1', 'P4'],
    'Nombre': ['Manzana', 'Cereza'],
    'Precio': [1.00, 1.25]
})

In [5]:
pd.concat([df_productos, df_ventas], axis=0, ignore_index=True)

,ID_Producto,Nombre,Precio
0,P1,Manzana,1.00
1,P2,Banana,0.50
2,P3,Cereza,0.75
3,P4,Durazno,1.25
4,P1,Manzana,1.00
5,P4,Cereza,1.25


# Reshaping, sorting, transposing

1. `nlargest`
2. `nsmallest`
3. `stack`
4. `unstack`
5. `pivot`
6. `pivot_table`

## pivot

[link](https://www.w3resource.com/pandas/dataframe/dataframe-pivot.php)

**Returns:** DataFrame
Returns reshaped DataFrame.

**Raises:** ValueError- Cuando hay algún índice, combinaciones de columnas con múltiples valores, es decir, un único valor por cada combinación de pares index-columns. Por ejemplo no prodría haber dos paises Argentina con la misma letra A que tengan valores diferentes.

In [127]:
df = pd.DataFrame({'pais': ['Argentina', 'Bolivia', 'Bolivia', 'Argentina', 'Argentina',
                           'Bolivia'],
                   'letra': ['A', 'B', 'C', 'C', 'B', 'A'],
                   'altura': [1, 2, 3, 4, 5, 6],
                   'name': ['h', 'i', 'j', 'k', 'l', 'm']})
df

,pais,letra,altura,name
0,Argentina,A,1,h
1,Bolivia,B,2,i
2,Bolivia,C,3,j
3,Argentina,C,4,k
4,Argentina,B,5,l
5,Bolivia,A,6,m


In [128]:
df.pivot(index='pais', columns='letra', values='altura')

letra,A,B,C
pais,,,
Argentina,1,5,4
Bolivia,6,2,3


In [129]:
df.pivot(index='pais', columns='letra')

altura       name      
letra          A  B  C    A  B  C
pais                             
Argentina      1  5  4    h  l  k
Bolivia        6  2  3    m  i  j

## Stack

In [130]:
df.stack()

0  pais      Argentina
   letra             A
   altura            1
   name              h
1  pais        Bolivia
   letra             B
   altura            2
   name              i
2  pais        Bolivia
   letra             C
   altura            3
   name              j
3  pais      Argentina
   letra             C
   altura            4
   name              k
4  pais      Argentina
   letra             B
   altura            5
   name              l
5  pais        Bolivia
   letra             A
   altura            6
   name              m
dtype: object

- index repetidos. Tener cuidado con los indices repetidos.

In [131]:
df.set_index(["pais"]).stack()

pais             
Argentina  letra     A
           altura    1
           name      h
Bolivia    letra     B
           altura    2
           name      i
           letra     C
           altura    3
           name      j
Argentina  letra     C
           altura    4
           name      k
           letra     B
           altura    5
           name      l
Bolivia    letra     A
           altura    6
           name      m
dtype: object

## Unstack

- "Columns with duplicate values are not supported in stack" los valores duplicados en las columnas no son soportados en stack index.

In [132]:
a = df#.set_index(['name'])  # con "pais" se lanza un error
a

,pais,letra,altura,name
0,Argentina,A,1,h
1,Bolivia,B,2,i
2,Bolivia,C,3,j
3,Argentina,C,4,k
4,Argentina,B,5,l
5,Bolivia,A,6,m


In [133]:
a.unstack()

pais    0    Argentina
        1      Bolivia
        2      Bolivia
        3    Argentina
        4    Argentina
        5      Bolivia
letra   0            A
        1            B
        2            C
        3            C
        4            B
        5            A
altura  0            1
        1            2
        2            3
        3            4
        4            5
        5            6
name    0            h
        1            i
        2            j
        3            k
        4            l
        5            m
dtype: object

# Selección de columnas

In [134]:
df

,pais,letra,altura,name
0,Argentina,A,1,h
1,Bolivia,B,2,i
2,Bolivia,C,3,j
3,Argentina,C,4,k
4,Argentina,B,5,l
5,Bolivia,A,6,m


In [135]:
fer = lambda x: x.altura % 2 == 0

In [136]:
fer(df)

0    False
1     True
2    False
3     True
4    False
5     True
Name: altura, dtype: bool

In [137]:
df.iloc[fer(df).values]

,pais,letra,altura,name
1,Bolivia,B,2,i
3,Argentina,C,4,k
5,Bolivia,A,6,m


In [138]:
df.set_index("pais", inplace=True)

In [139]:
df

,letra,altura,name
pais,,,
Argentina,A,1,h
Bolivia,B,2,i
Bolivia,C,3,j
Argentina,C,4,k
Argentina,B,5,l
Bolivia,A,6,m


In [140]:
import pandas as pd

# Creamos un DataFrame de ejemplo
data = {'Nombre': ['Juan', 'María', 'Pedro', 'Ana'],
        'Edad': [25, 32, 18, 41]}
df = pd.DataFrame(data)

# Definimos una función que usaremos con map
def categorizar_edad(edad, adolescente=18, adulto=30):
    if edad < adolescente:
        return 'Menor'
    elif edad < adulto:
        return 'Adolescente'
    else:
        return 'Adulto'

# Aplicamos la función a la columna 'Edad' usando map y pasando kwargs

print(df)   # Deveria imprimir


# solucion
# Aplicamos la función a la columna 'Edad' usando map y pasando kwargs con una función lambda
df['Categoría'] = df['Edad'].map(lambda x: categorizar_edad(x, adolescente=20, adulto=35))

  Nombre  Edad
0   Juan    25
1  María    32
2  Pedro    18
3    Ana    41


In [141]:
df

,Nombre,Edad,Categoría
0,Juan,25,Adolescente
1,María,32,Adolescente
2,Pedro,18,Menor
3,Ana,41,Adulto


In [142]:
df.agg(['sum', 'min'])

,Nombre,Edad,Categoría
sum,JuanMaríaPedroAna,116,AdolescenteAdolescenteMenorAdulto
min,Ana,18,Adolescente
